# Up and running with PANDA and netZooPy

### Author:
Daniel Morgan*.

*Channing division of network medicine, Brigham's and Women hospital and Harvard Medical School, Boston, MA.

## Introduction
Regulatory network reconstruction is a fundamental problem in computational biology. There are significant limitations to such reconstruction using individual datasets, and increasingly people attempt to construct networks using multiple, independent datasets obtained from complementary sources, but methods for this integration are lacking. We developed PANDA (Passing Attributes between Networks for Data Assimilation), a message-passing model using multiple sources of information to predict regulatory relationships, and used it to integrate protein-protein interaction, gene expression, and sequence motif data to reconstruct genome-wide, condition-specific regulatory networks in yeast as a model. The resulting networks were not only more accurate than those produced using individual data sets and other existing methods, but they also captured information regarding specific biological mechanisms and pathways that were missed using other methodologies. PANDA is scalable to higher eukaryotes, applicable to specific tissue or cell type data and conceptually generalizable to include a variety of regulatory, interaction, expression, and other genome-scale data.

PANDA starts with a prior network of putative regulatory interactions (center network in the image below), a prior network of protein-protein interactions between transcription factors, and target gene expression data, which is converted into a co-expression network.

<img src="../../img/panda.png" style="width: 200px;">  

A message passing framework is used to find agreement between the three input networks. First, the responsibility (R) is calculated: 

<img src="../../img/responsibility.png" style="width: 200px;">  

as well as the availability (A): 

<img src="../../img/availability.png" style="width: 200px;">  

The prior gene regulatory network W is then updated using the responsibility and availability:  

<img src="../../img/combine.png" style="width: 300px;">  

Next, the protein cooperativity and gene co-regulatory networks are updated::

<img src="../../img/cooperativity.png" style="width: 300px;">  
<img src="../../img/co-regulatory.png" style="width: 300px;"> 

Self-interactions in P and C are also updated to satisfy convergence:  

<img src="../../img/p.png" style="width: 300px;">  
<img src="../../img/c.png" style="width: 300px;">  

, which is evaluated using a hamming distance:

<img src="../../img/hamming.png" style="width: 300px;">  

In [ ]:
from IPython.display import Image
from IPython.core.display import HTML
Image(url= "https://journals.plos.org/plosone/article/figure/image?size=large&id=info:doi/10.1371/journal.pone.0064832.g001", width=500, height=500)

## 1. Installation and Setup

In [ ]:
%%bash
cd ~
# git clone https://github.com/netZoo/netZooPy.git
cd netZooPy
pip3 install -e .

or with conda

In [ ]:
# source activate myenv
# conda install git pip
# pip install git+git://github.com/netZoo/netZooPy.git

In [ ]:
import os
# os.getcwd()
# os.chdir("~/")
os.getcwd()

In [ ]:
from netZooPy.panda.panda import Panda
import pandas as pd
import matplotlib.pyplot as plt

## 2. Parameter Setting & Exploring the Data

First, we start by setting the path to the 1) motif prior network, 2) the gene expression data, and 3) the ppi network data.
The motif prior network is typically a TF-by-gene binary matrix where 1 indicates the presence of sequence (motif) of a TF in the gene regulatory region and 0 otherwise.
Gene expression data is typically a gene-by-sample matrix containing expression data.
PPI network is a TF-by-TF binary matrix, where 1 indicates a physical interaction between two TFs and 0 otherwise.
If two TFs are likely to binding, they are likely to form regulatory complexes for the same genes.

In [ ]:
expression_data='netZooPy/tests/ToyData/ToyExpressionData.txt'
motif_data='netZooPy/tests/ToyData/ToyMotifData.txt'
ppi_data='netZooPy/tests/ToyData/ToyPPIData.txt'
panda_output='netZooPy/tests/panda/output_panda.txt'

There are 1000 genes and 51 samples in our toy data. This is your novel input. The remaining files are known interaction lists.

In [ ]:
motif_data=pd.read_csv(motif_data,sep="\t",header=None)
motif_data[0].unique().size

In [ ]:
motif_data[1].unique().size

Since the first column is TF, you thus have 87 TF and 913 genes are returned from the second column, with their interaction weights in the third column (motif_data[2]). Now lets check out the ppi data, another interaction list with three columns, with 238 interactions between the TF.

In [ ]:
ppi_data=pd.read_csv(ppi_data,sep="\t",header=None)
ppi_data.shape

## 3. Calling PANDA

One can chose to run in terminal simply by pointing to the input files

In [ ]:
# %%bash
# cd netZooPy
# pip3 install -e .
# python netZooPy/panda/run_panda.py -e netZooPy/tests/ToyData/ToyExpressionData.txt -m netZooPy/tests/ToyData/ToyMotifData.txt -p netZooPy/tests/ToyData/ToyPPIData.txt -f True -o test_panda.txt

Alternatively one can continue running in Jupyter, using all data sources:

In [ ]:
expression_data='netZooPy/tests/ToyData/ToyExpressionData.txt'
motif_data='netZooPy/tests/ToyData/ToyMotifData.txt'
ppi_data='netZooPy/tests/ToyData/ToyPPIData.txt'
panda_obj = Panda(expression_data, motif_data, ppi_data, save_tmp=True,save_memory = False, remove_missing=False, keep_expression_matrix = False)
panda_obj.save_panda_results(panda_output)

In [ ]:
import sys
sys.getsizeof(panda_obj)

In [ ]:
panda_obj.top_network_plot(top=10)

using only the motif prior

In [ ]:
expression_data=None
motif_data='netZooPy/tests/ToyData/ToyMotifData.txt'
ppi_data=None
panda_obj = Panda(expression_data,  motif_data, ppi_data,remove_missing=True)
panda_obj.save_panda_results(panda_output)

In [ ]:
panda_obj.top_network_plot(top=10)

without the expression matrix

In [ ]:
expression_data=None
motif_data='netZooPy/tests/ToyData/ToyMotifData.txt'
ppi_data='netZooPy/tests/ToyData/ToyPPIData.txt'
panda_obj = Panda(expression_data,  motif_data, ppi_data,remove_missing=True)
panda_obj.save_panda_results(panda_output)

In [ ]:
panda_obj.top_network_plot(top=10)

and without using a motif prior

In [ ]:
expression_data='netZooPy/tests/ToyData/ToyExpressionData.txt'
motif_data=None
ppi_data='netZooPy/tests/ToyData/ToyPPIData.txt'
panda_obj = Panda(expression_data, motif_data, ppi_data)
panda_obj.save_panda_results(panda_output)

You can also save the memory by deleting intermediary variables by using `save_memory=True`. However, we will compute the gene indegree, therefore we need to keep those variables in the object by setting `save_memory=False`.

In [ ]:
expression_data='netZooPy/tests/ToyData/ToyExpressionData.txt'
motif_data='netZooPy/tests/ToyData/ToyMotifData.txt'
ppi_data='netZooPy/tests/ToyData/ToyPPIData.txt'
panda_obj = Panda(expression_data, motif_data, ppi_data, save_memory=True)
panda_obj.save_panda_results(panda_output)

Basic follow up analysis is also possible, such as degree calculation per gene

In [ ]:
panda_obj.return_panda_indegree()

panda_obj.save_panda_results()

## 4. References
Glass K, Huttenhower C, Quackenbush J, Yuan GC. Passing Messages Between Biological Networks to Refine Predicted Interactions, PLoS One, 2013 May 31;8(5):e64832